<a href="https://colab.research.google.com/github/TurkuNLP/Turku-paraphrase-models/blob/main/paraphrase_model_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, install all necessary dependencies and download the models and the associated code.

In [ ]:
!pip install transformers
!pip install pytorch-lightning
!git clone https://github.com/TurkuNLP/paraphrase-classification para
!wget http://dl.turkunlp.org/finbert/torch-transformers/bert-large-finnish-cased-rc2/bert-large-finnish-cased-rc2.tar.gz
!gzip -d bert-large-finnish-cased-rc2.tar.gz

fatal: destination path 'paraphrase-classification' already exists and is not an empty directory.
--2021-08-05 12:42:48--  http://dl.turkunlp.org/finbert/torch-transformers/bert-large-finnish-cased-rc2/bert-large-finnish-cased-rc2.tar.gz
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1322927291 (1.2G) [application/octet-stream]
Saving to: ‘bert-large-finnish-cased-rc2.tar.gz’

bert-large-finnish- 100%[===================>]   1.23G  14.4MB/s    in 82s     

2021-08-05 12:44:11 (15.3 MB/s) - ‘bert-large-finnish-cased-rc2.tar.gz’ saved [1322927291/1322927291]



Then set up the model and write some helper functions.

In [ ]:
import torch
import transformers
from para.notebook import para_model

bert_model = "bert-large-finnish-cased-rc2"

model = para_model.ParaMultiOutputAvgModel.load_from_checkpoint(checkpoint_path="/scratch/project_2002820/rastasii/checkpoints/train-alpha2-combined-neg-autoneg-1.0-final/model.ckpt", bert_model=bert_model)

model.eval()
model.cuda()

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

def compute_masks(mask):
    one_idx = [i for i, b in enumerate(mask) if b]
    zeros = torch.zeros(len(mask), dtype=torch.long)
    cls_mask, sep1_mask, sep2_mask = [torch.scatter(zeros, 0, torch.tensor(i), torch.tensor(1)) for i in one_idx]
    left_idx = torch.tensor(range(one_idx[0]+1, one_idx[1]))
    left_mask = torch.scatter(zeros, 0, left_idx, torch.ones(len(left_idx), dtype=torch.long))
    right_idx = torch.tensor(range(one_idx[1]+1, one_idx[2]))
    right_mask = torch.scatter(zeros, 0, right_idx, torch.ones(len(right_idx), dtype=torch.long))
    return {'cls_mask': cls_mask, 'sep1_mask': sep1_mask, 'sep2_mask': sep2_mask, 'left_mask': left_mask, 'right_mask': right_mask}

def encode(tokenizer, txt1, txt2):
    t1_tok=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(txt1))
    t2_tok=tokenizer.convert_tokens_to_ids(tokenizer.tokenize(txt2))
    encoded=tokenizer.prepare_for_model(t1_tok, t2_tok, return_length=True, return_special_tokens_mask=True, max_length=512, truncation=True, padding='longest', return_tensors='pt')
    r = {"input_ids":encoded.input_ids, "token_type_ids":encoded.token_type_ids, "attention_mask":encoded.attention_mask, "length":encoded.length, **compute_masks(encoded.special_tokens_mask)}
    for k in "input_ids", "token_type_ids", "attention_mask", "cls_mask", "sep1_mask", "sep2_mask", "left_mask", "right_mask":
        r[k] = torch.unsqueeze(r[k], 0).to(device='cuda')
    return r

FileNotFoundError: ignored

Finally, run some paraphrase candidates through the model.

In [ ]:
with torch.no_grad():
    # r = model(encode(tokenizer, "Tämä kakku on herkullista.", "Onpa hyvää kakkua."))
    r = model(encode(tokenizer, "Tämäpä vasta oli hienoa.", "Se oli ihan sairaan siistiä."))
    indices = {k: torch.argmax(v) for k, v in r.items()}
    flag_i2lab = {k: {vd: kd for kd, vd in v.items()} for k, v in model.flag_lab2i.items()}
    label_dict = {k: flag_i2lab[k][v.item()] for k, v in indices.items()}
    label = label_dict['base'] + (label_dict['direction'] if label_dict['direction'] != 'None' else '') + ('i' if label_dict['has_i'] else '') + ('s' if label_dict['has_s'] else '')
    print(label)